### 作業目標: 了解N-Gram如何藉由文本計算機率

### 為何需要使用馬可夫假設來簡化語言模型的計算?

原本的語言模型利用貝氏定理計算機率時為:
$$
\begin{aligned}
&W = (W_1W_2W_3W_4…W_m) \\
&P(W_1,W_2,W_3,W_4,…,W_m) = P(W_1)*P(W_2|W_1)*P(W_3|W_1,W_2)*…*P(W_m|W_1,…,W_{m-1})
\end{aligned}
$$

為何需要引入馬可夫假設使機率簡化為:
$$
P(W_m|W_1,W_2,W_3,…,W_{m-1}) = P(W_m|W_{m-n+1},W_{m-n+2},…,W_{m-1})
$$

### 	<font color=#FF0000>Answer</font>

基本上，一個文字的出現跟前幾個字的序列有關係，所以必須引入馬可夫模型。 <br>
然而一個完整的馬可夫模型，需要列舉所有狀態的條件機率。 <br>
但是，語言有所謂的詞頻，所以可以將馬可夫模型簡化到n階馬可夫模型(n=2,3,4...)。 <br>
當n越大對文字約束力越大，但複雜度會越高。 <br>
基本上，因為中文的2、3字詞較多，固可使用2~3階馬可夫模型來簡化機率。 <br>

### 以Bigram模型下判斷語句是否合理

已知的機率值有
1. p(i|_start_) = 0.25
2. p(english|want) = 0.0011
3. p(food|english) = 0.5
4. p(_end_|food) = 0.68
5. P(want|_start_) = 0.25
6. P(english|i) = 0.0011

請根據已給的機率與所計算出的機率(df_bigram_prob), 試著判斷下列兩個句子哪個較為合理

s1 = “i want english food”

s2 = "want i english food"

In [1]:
dict_prob={'p_i_start': 0.25, 
           'p_english_want':0.0011, 
           'p_food_english': 0.5, 
           'p_end_food': 0.68, 
           'p_want_start': 0.25, 
           'p_english_i': 0.0011,
           'p_want_i': 0.002157,
           'p_i_want': 0.32649}
s1 = 'i want english food'
s2 = 'want i english food'

In [2]:
from functools import reduce

for i,s in enumerate([s1,s2]):
    words = s.split()
    prob = [dict_prob['p_'+words[0]+'_start']]
    
    print(f's{i+1}:  {s: <20}\n\n  P({words[0]}|start) * ', end='')
    for pair in zip(words,words[1:]):
        try:
            prob += [dict_prob['p_'+pair[0]+'_'+pair[1]]]
            print(f'P({pair[0]}|{pair[1]}) * ', end = '')
        except:
            prob += [dict_prob['p_'+pair[1]+'_'+pair[0]]]
            print(f'P({pair[1]}|{pair[0]}) * ', end = '')
    
    prob += [dict_prob['p_end_'+words[-1]]]
    print(f'P(end|{words[-1]})\n= {" * ".join(list(map(str,prob)))}\n= {reduce(lambda x,y : x * y, prob):<.12f}')
    print('' if i else '\n'*2, end='')

s1:  i want english food 

  P(i|start) * P(i|want) * P(english|want) * P(food|english) * P(end|food)
= 0.25 * 0.32649 * 0.0011 * 0.5 * 0.68
= 0.000030526815


s2:  want i english food 

  P(want|start) * P(want|i) * P(english|i) * P(food|english) * P(end|food)
= 0.25 * 0.002157 * 0.0011 * 0.5 * 0.68
= 0.000000201680


### 了解N-Gram如何藉由文本計算機率

In [3]:
import numpy as np
import pandas as pd
words = 'i want to eat chinese food lunch spend'.split()
word_cnts = np.array([2533, 927, 2417, 746, 158, 1093, 341, 278]).reshape(1, -1)
df_word_cnts = pd.DataFrame(word_cnts, columns=words)
print(df_word_cnts)

      i  want    to  eat  chinese  food  lunch  spend
0  2533   927  2417  746      158  1093    341    278


In [4]:
# 記錄當前字與前一個字詞的存在頻率
bigram_word_cnts = [[5, 827, 0, 9, 0, 0, 0, 2], [2, 0, 608, 1, 6, 6, 5, 1], [2, 0, 4, 686, 2, 0, 6, 211],
                    [0, 0, 2, 0, 16, 2, 42, 0], [1, 0, 0, 0, 0, 82, 1, 0], [15, 0, 15, 0, 1, 4, 0, 0],
                    [2, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0]]

df_bigram_word_cnts = pd.DataFrame(bigram_word_cnts, columns=words, index=words)
print(df_bigram_word_cnts)

          i  want   to  eat  chinese  food  lunch  spend
i         5   827    0    9        0     0      0      2
want      2     0  608    1        6     6      5      1
to        2     0    4  686        2     0      6    211
eat       0     0    2    0       16     2     42      0
chinese   1     0    0    0        0    82      1      0
food     15     0   15    0        1     4      0      0
lunch     2     0    0    0        0     1      0      0
spend     1     0    1    0        0     0      0      0


由上表可知當前一個字詞(列)是want的時候, 當前字詞(行)是to的頻率在文本中共有608次

In [5]:
#請根據給出的總詞頻(df_word_cnts)與bigram模型的詞頻(df_bigram_word_cnts)計算出所有詞的配對機率(ex:p(want|i))
df_bigram_prob = df_bigram_word_cnts.copy()
for word in words:
    df_bigram_prob.loc[word, :] = df_bigram_word_cnts.loc[word, :] / df_word_cnts.loc[0, word]
print(df_bigram_prob)

                i     want        to       eat   chinese      food     lunch  \
i        0.001974  0.32649  0.000000  0.003553  0.000000  0.000000  0.000000   
want     0.002157  0.00000  0.655879  0.001079  0.006472  0.006472  0.005394   
to       0.000827  0.00000  0.001655  0.283823  0.000827  0.000000  0.002482   
eat      0.000000  0.00000  0.002681  0.000000  0.021448  0.002681  0.056300   
chinese  0.006329  0.00000  0.000000  0.000000  0.000000  0.518987  0.006329   
food     0.013724  0.00000  0.013724  0.000000  0.000915  0.003660  0.000000   
lunch    0.005865  0.00000  0.000000  0.000000  0.000000  0.002933  0.000000   
spend    0.003597  0.00000  0.003597  0.000000  0.000000  0.000000  0.000000   

            spend  
i        0.000790  
want     0.001079  
to       0.087298  
eat      0.000000  
chinese  0.000000  
food     0.000000  
lunch    0.000000  
spend    0.000000  
